In [24]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black


In [25]:
import requests as rq
import json
import pandas as pd

In [26]:
USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36"
HEADERS = {"User-Agent": USER_AGENT}

In [27]:
URL = "https://www.zapimoveis.com.br/venda/casas/?pagina="
pages = [str(i) for i in range(1, 3)]

In [31]:
def process_json(json_dict):
    results_listining = json_dict["results"]["listings"]
    for i, house in enumerate(results_listining):
        parking_spaces = results_listining[i]["listing"]["parkingSpaces"]
        results_listining[i]["listing"]["parkingSpaces"] = (
            parking_spaces[0] if parking_spaces else 0
        )
        suites = results_listining[i]["listing"]["suites"]
        results_listining[i]["listing"]["suites"] = suites[0] if suites else 0
        bathrooms = results_listining[i]["listing"]["bathrooms"]
        results_listining[i]["listing"]["bathrooms"] = bathrooms[0] if bathrooms else 0
        bedrooms = results_listining[i]["listing"]["bedrooms"]
        results_listining[i]["listing"]["bedrooms"] = bedrooms[0] if bedrooms else 0
        usable_areas = results_listining[i]["listing"]["usableAreas"]
        results_listining[i]["listing"]["usableAreas"] = int(usable_areas) if usable_areas else 0
        
    json_dict["results"]["listings"] = results_listining
    return json_dict

In [32]:
def get_house_info(html: str):
    json_start = html.find('"results":{"listings"')
    json_end = html.find(";(function(){var s")

    json_str = "{" + html[json_start:json_end]

    json_dict = json.loads(json_str)
    json_dict = process_json(json_dict)
    flattened_json = pd.json_normalize(json_dict["results"]["listings"])
    print(flattened_json)
    return flattened_json


def get_page_info(url: str, pages: list, columns: list):
    result = []
    for page in pages:
        response = rq.get(URL + page, headers=HEADERS)
        if response.status_code == 200:
            html = str(response.text)
            result.append(get_house_info(html))

    data = pd.concat(result, ignore_index=True)
    if columns:
        data = data[columns]

    return data

In [33]:
raw_data = get_page_info(URL, pages, [])

            type link.data.streetNumber link.data.state    link.data.city   
0   superPremium                              São Paulo    Ribeirão Preto  \
1   superPremium                    560  Espírito Santo             Serra   
2   superPremium                    186          Paraná           Colombo   
3            NaN                     03           Bahia  Feira de Santana   
4            NaN                    792           Bahia  Feira de Santana   
..           ...                    ...             ...               ...   
95           NaN                      1           Goiás           Goiânia   
96           NaN                     60           Bahia  Feira de Santana   
97           NaN                     02           Bahia  Feira de Santana   
98           NaN                   1000  Espírito Santo             Viana   
99           NaN                   1605       São Paulo          Mongaguá   

   link.data.zone              link.data.neighborhood   
0         Bairros 

In [34]:
raw_data.to_csv(f"raw_data_{len(raw_data)}.csv", index=False)